# GalaxieSQL

## Description du projet

Atelier SQL parents enfants: enseigner les bases de SQL (similaire à AccioQuery mais moins de détails, on veut seulement: select,*, from, where, AND, OR, NOT, LIKE) mais aussi comment entrer un nouveau élément dans la base de données: ``INSERT INTO ... VALUES ...``. Pour cela le pitch:
> Vous êtes des scientifiques de l'espace et vous vous intéressez aux habitants de planêtes habitées récemment découvertes. Trouvez le nom d'un extraterestre qui n'as pas encore été étudié grâce à la base de données. (on leur dira de noter le nom sur un post-it et le coller sur le poster lorsqu'elle l'auront trouvé). Quelques extraterrestres n'ont pas encore été ajoutés dans la base de données (on les notes avec des points d'interrogation), ajoutez les et assurez vous ensuite qu'ils ont bien été ajoutés à la base de données. 

Dans un second temps, on pourrait aussi leur proposer de générer leur propre extraterrestre (c.f. génération d'image d'alien à partir de description). Pour cette seconde partie, il faudrait avoir un moyen de lancer la fonction.

In [55]:
import sqlite3 # gérer les bases de données
import pickle # sauvegarder des structures de données python 
import csv # pour lire et modifier les fichiers csv
import random # pour générer des nombres aléatoires
from PIL import Image # Pour afficher et modifier les images
from IPython.display import SVG
from aliens import generate_alien,change_colour,draw_alien, dessine_alien as draw_alien_svg # fonction implémentées dans aliens.py utile pour la génération d'aliens et le changement de couleur
import os
print(os.getcwd()) #current working directory

/home/mischa/d/Meetup


## Génération des attributs
On commence par définir les attributs que l'on veut pour nos aliens. On fait des listes pour certains attributs et on utilise des liste csv externe pour les noms.

In [56]:
palette_RGB = {"bleu":(0,0,255), "gris":(100,100,100), "noir":(0,0,0),"violet":(128, 0, 255),
           "rouge":(255,0,0),"blanc":(255,255,255),"vert":(0,255,0),"jaune":(255,255,0),
           "rose":(255, 0, 255),"orange":(255,150,0),"marron":(150,100,50)}
palette = {"bleu":"#0000ff", "gris":"#646464", "noir":"#000000","violet":"#8000ff",
           "rouge":"#ff0000","blanc":"#ffffff","vert":"#00ff00","jaune":"#ffff00",
           "rose":"#ff00ff","orange":"#ff9000","marron":"#64503c"}
palette_RGB_de = {"blau":(0,0,255), "grau":(100,100,100), "schwarz":(0,0,0),"violett":(128, 0, 255),
           "rot":(255,0,0),"weiss":(255,255,255),"grün":(0,255,0),"gelb":(255,255,0),
           "rosa":(255, 0, 255),"orange":(255,150,0),"braun":(150,100,50)}
palette_de = {"blau":"#0000ff", "grau":"#646464", "schwarz":"#000000","violett":"#8000ff",
           "rot":"#ff0000","weiss":"#ffffff","grün":"#00ff00","gelb":"#ffff00",
           "rosa":"#ff00ff","orange":"#ff9000","braun":"#64503c"}
attributs = dict({"nom" : [],
"couleurs" : palette,
"couleurs_RGB": palette_RGB,
"farben" : palette_de,
"farben_RGB": palette_RGB_de,
"planète" : ["céplusplus", "scratchon", "pythos"],
"visage" : ["trompe","bec","museau","rien","nez","bouche","corne"],
"Gesicht" : ["Rüssel","Schnabel", "Schnauze","nichts","Nase","Mund","Horn"],
"tête" : ["ronde","ovale","carrée","triangulaire","difforme"],
"Kopf" : ["rund","oval","rechteckig","dreieckig","unförmig"],
"peau" : ["poils","rien","écailles","épines","plumes","carapace"],
"Haut" : ["Haare", "nichts", "Schuppen", "Stachel", "Federn", "Panzer"],
"ingrédients" : ["à l\'aubergine","au olives","au poivrons","à la mozarella","à la burrata","au gorgonzola","Gruyère","au poivre",
                 "au sel","à l\'huile d\'olive","à l\'ananas", "au sugus","à la fraise","au chocolat"],
"Zutaten" : ["Aubergine", "Oliven", "Paprika", "Mozarella", "Burrata", "Gorgonzola", "Gruyère", "Pfeffer",
            "Saltz", "Olivenöl", "Ananas", "Sugus", "Erdbeeren", "Schokolade"]
}) 

In [57]:
noms_latin=[]
with open('noms_latin2.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for line in spamreader:
        noms_latin.append(line[0])
attributs["nom"] = noms_latin[1:]
attributs["nom"].extend(['BASICUS','COBOLUS','COQUS','CURLUS','FORTRUS','GONUM', 'HASKELLION', 'JAVIA',
                        'KOTLINI','LISPUS','MATLABUM','OCAMLIUM','PASCALANS','TEX','WOLFRAM'])
print(attributs["couleurs"])

{'bleu': '#0000ff', 'gris': '#646464', 'noir': '#000000', 'violet': '#8000ff', 'rouge': '#ff0000', 'blanc': '#ffffff', 'vert': '#00ff00', 'jaune': '#ffff00', 'rose': '#ff00ff', 'orange': '#ff9000', 'marron': '#64503c'}


In [58]:
# On sauvegarde le tout pour ré-ouvrir plus tard dans aliens.py
with open('attributs_aliens.pkl', 'wb') as f:
    pickle.dump(attributs, f)

In [59]:
# Pas indispensable mais pour vérifier que l'on a bien enregistré le tout
with open('attributs_aliens.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)
print(loaded_dict)

{'nom': ['ACER', 'ACHILLEA', 'ACHNATHERUM', 'ACONITUM', 'ACTINIDIA', 'ADIANTHUM', 'AEGOPODIUM', 'AESCULUS', 'AJUGA', 'ALCHEMILLA', 'ALNUS', 'ALTHAE', 'AMORPHA', 'ANDROMEDA', 'AQUILEGIA', 'ARABIS', 'ARA\xa0LIA', 'ARCTOSTAPHYLOS', 'ARENARIA', 'ARISAEMA', 'ARMERIA', 'ARONIA', 'ARTEMISIA', 'ARUNCUS', 'ASARUM', 'ASIMINA', 'ASTILBOIDES', 'ASTRANTIA', 'AZALEA', 'BERBERIS', 'BETULA', 'BRUNNERA', 'BUXUS', 'CALLUNA', 'CAMPANULA', 'CARAGANA', 'CAREX', 'CARPINUS', 'CARYA', 'CASSANDRA', 'CELASTRUS', 'CELTIS', 'CENTAUREA', 'CERASTIUM', 'CENTRANTHUS', 'CEPHALANTUS', 'CHAENOMELES', 'CHAMAECYPARIS', 'CHIONANTHUS', 'CHRYSANTHEMUM', 'CIMICIFUGA', 'CLEMATIS', 'CLETHRA', 'COMPTONIA', 'CONVALLARIA', 'CORNUS', 'CORYLUS', 'COTINUS', 'CRATAEGUS', 'CYTISUS', 'DELPHINIUM', 'DIANTHUS', 'DICENTRA', 'DIGITALIS', 'DIRCA', 'DIERVILLA', 'DRYOPTERIS ', 'ECHINACEA', 'ECHINOPS', 'ELEAGNUS', 'ELEUTHEROCOCCUS', 'EMPETRUM', 'EPIMEDIUM', 'ERYNGIUM', 'EUONYMUS', 'EUPATORIUM', 'EUPHORBIA', 'FAGUS', 'FALLOPIA', 'FESTUCA', 'FILI

## Génération de la base de données 
On se connecte à la base de donnée avec un curseur par lequel on peut faire passer les commandes sur la base de données.

In [60]:
connection = sqlite3.connect("aliens_fr.db")
connection_de = sqlite3.connect("alien_de.db")
cursor = connection.cursor()
cursor_de = connection_de.cursor()

On crée un tableau dans la base de données. Ceci doit seulement être fait la première fois que l'on crée le tableau.

In [63]:
cursor.execute(
    "CREATE TABLE extraterrestres (nom TEXT NOT NULL PRIMARY KEY, \
    planète TEXT, \
    nombre_membres INT, \
    taille INT, \
    nombre_yeux INT, couleur_yeux TEXT, \
    peau TEXT, couleur_peau TEXT,\
    antennes INT, visage TEXT, tête TEXT, \
    pizza TEXT)")
cursor_de.execute(
    "CREATE TABLE Ausserirdische (Name TEXT NOT NULL PRIMARY KEY, \
    Planet TEXT, \
    Anzahl_Koerperglieder INT, \
    Groesse INT, \
    anzahl_Augen INT, farbe_Augen TEXT, \
    Haut TEXT, farbe_Haut TEXT,\
    Antennen INT, Gesicht TEXT, Kopf TEXT, \
    Pizza TEXT)")

OperationalError: table extraterrestres already exists

On va ensuite peupler la base de données. Pour cela, on utilise la fonction ``` generate_alien() ``` définie dans le fichier alien.py. On veut que les noms des aliens soient uniques (et par extension leur attributs sont uniques) donc on ignore les doublons. 

In [64]:
nAliens = 9000
for alien in range(nAliens):
    attributs_fr, attributs_de = generate_alien()
    try:
        cursor.execute(f'Insert INTO extraterrestres VALUES("{attributs_fr[0]}",\
"{attributs_fr[1]}",{attributs_fr[2]},{attributs_fr[3]},{attributs_fr[4]},\
"{attributs_fr[5]}","{attributs_fr[6]}","{attributs_fr[7]}",{attributs_fr[8]},\
"{attributs_fr[9]}","{attributs_fr[10]}","{attributs_fr[11]}")')
        cursor_de.execute(f'Insert INTO Ausserirdische VALUES("{attributs_de[0]}",\
"{attributs_de[1]}",{attributs_de[2]},{attributs_de[3]},{attributs_de[4]},\
"{attributs_de[5]}","{attributs_de[6]}","{attributs_de[7]}",{attributs_de[8]},\
"{attributs_de[9]}","{attributs_de[10]}","{attributs_de[11]}")')
    except:
        #print(f"{attributs_fr[0]} existe déjà")
        continue

On peut tester les bases de donnée ici:

In [65]:
# Nom des colones
# print(cursor.execute("PRAGMA table_info(extraterrestres)").fetchall())
# print(cursor.execute(f"SELECT * from extraterrestres LIMIT 3").fetchall())
print(cursor.execute("SELECT * from extraterrestres LIMIT 2").fetchall())
print(cursor_de.execute("SELECT * from Ausserirdische LIMIT 2").fetchall())

[('ROBINA_5340', 'pythos', 5, 3395, 4, 'rouge', 'plumes', 'rose', 0, 'trompe', 'difforme', "Pizza au poivre, à la burrata, à l'ananas, au sel, et au gorgonzola."), ('FILIPENDULA_1481', 'pythos', 1, 4436, 8, 'orange', 'écailles', 'noir', 1, 'corne', 'difforme', 'Pizza à la fraise')]
[('ROBINA_5340', 'pythos', 5, 3395, 4, 'rot', 'Federn', 'rosa', 0, 'Rüssel', 'unförmig', 'Pizza mit Aubergine, und Ananas.'), ('FILIPENDULA_1481', 'pythos', 1, 4436, 8, 'orange', 'Schuppen', 'schwarz', 1, 'Horn', 'unförmig', 'Pizza mit Gruyère, Gorgonzola, Ananas, und Oliven.')]


Enfin, il faut toujours sauvegarder les changements dans la base de données puis rompre la connection.

In [66]:
#sauver les changements
connection.commit()
connection_de.commit()
#fermer la connection à la base de données
cursor.close()
cursor_de.close()
connection.close()
connection_de.close()

### Génération d'image d'alien à partir de description
On essaie de générer des dessins d'aliens à partir de leur description. Pour cela on prépare différentes versions des différents aliens avec adobe illustrator. Rester dans les délimitations pour les différentes parties pour que ça match. Fichier->Exporter->Exporter sous-> cocher utiliser les plans de travail, Exporter en png. Puis on les combines. On peut donc créer un alien pour chaque entrée dans la base de données. Cela pourrait également être une autre partie dans l'atelier: créer son propre alien (plus en lien avec le design mais ça pourrait être amusant quand même).

La génération d'aliens est implémentée dans le document ```aliens.py``` et il y a un liste d'attributs à modifier.

In [67]:
# with open('merged.svg','r') as f:
#     svg_data = f.read()
# mod_image = svg_data.replace("#66b32e","#0000ff")
# SVG(mod_image)
for i in range(20):
    draw_alien_svg(filename = f"alien{i}.svg", save=True, peau=["écailles","poils","rien"][i%3])

FileNotFoundError: [Errno 2] No such file or directory: 'images/yeux_3.svg'

Faire une fonction ou on peut mettre les attributs pour générer les aliens. 
Afficher alien avec nom CORVUS_42 (va chercher attributs de corvus 42 et affiche).
Sauvegarder quelques images.

Important de savoir comment nommer les aliens

In [9]:
with open('merged.svg','r') as f:
    svg_data = f.read()
mod_image = svg_data.replace("#66b32e","#0000ff")
SVG(mod_image)

{(116, 171, 65, 255), (122, 149, 92, 255), (0, 0, 0, 48), (0, 0, 0, 112), (124, 156, 88, 255), (0, 0, 0, 176), (119, 142, 96, 255), (0, 0, 0, 240), (96, 104, 89, 255), (0, 0, 0, 255), (74, 78, 71, 255), (0, 0, 0, 32), (85, 91, 80, 255), (0, 0, 0, 96), (61, 62, 59, 255), (0, 0, 0, 160), (0, 0, 0, 224), (32, 32, 31, 255), (123, 161, 81, 255), (101, 179, 46, 255), (255, 255, 255, 0), (0, 0, 0, 16), (0, 0, 0, 80), (0, 0, 0, 144), (0, 0, 0, 208), (110, 175, 56, 255), (110, 125, 96, 255), (115, 134, 96, 255), (0, 0, 0, 64), (104, 116, 93, 255), (0, 0, 0, 128), (0, 0, 0, 192), (121, 167, 74, 255)}
